<a href="https://colab.research.google.com/github/patrickctrf/IA024_2022S2/blob/main/ex02_175480/ex02_175480_Patrick_Ferreira.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula 2: Análise de Sentimentos usando Bag of Words

Neste notebook iremos treinar um rede de uma única camada para fazer análise de sentimento usando o dataset IMDB.

In [ ]:
nome = "Patrick de Carvalho Tavares Rezende Ferreira"
print(f'Meu nome é {nome}')

Meu nome é Patrick de Carvalho Tavares Rezende Ferreira


# Importando as bibliotecas necessárias

In [ ]:
import collections
import pandas as pd
import re
import torch
from typing import List

# Preparando Dados

Primeiro, fazemos download do dataset:

In [ ]:
!wget -nc http://files.fast.ai/data/examples/imdb_sample.tgz
!tar -xzf imdb_sample.tgz

--2022-08-30 15:09:39--  http://files.fast.ai/data/examples/imdb_sample.tgz
Resolving files.fast.ai (files.fast.ai)... 104.26.2.19, 104.26.3.19, 172.67.69.159, ...
Connecting to files.fast.ai (files.fast.ai)|104.26.2.19|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://files.fast.ai/data/examples/imdb_sample.tgz [following]
--2022-08-30 15:09:39--  https://files.fast.ai/data/examples/imdb_sample.tgz
Connecting to files.fast.ai (files.fast.ai)|104.26.2.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 571827 (558K) [application/x-gtar-compressed]
Saving to: ‘imdb_sample.tgz’

imdb_sample.tgz     100%[===================>] 558.42K  --.-KB/s    in 0.03s   

2022-08-30 15:09:39 (21.4 MB/s) - ‘imdb_sample.tgz’ saved [571827/571827]



Carregamos o dataset .csv usando o pandas:

In [ ]:
df = pd.read_csv('imdb_sample/texts.csv')
df.shape
df.head()

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


Iremos agora apenas selecionar 100 exemplos de treinamento:

In [ ]:
treino = df[df['is_valid'] == False]  # Apenas treinamento, isto é, descartamos o dataset de validação.

print('treino.shape original:', treino.shape)

treino = treino[:100]  # Aqui truncamos o dataset para os 100 primeiros exemplos. 

print('treino.shape depois:', treino.shape)

treino.shape original: (800, 3)
treino.shape depois: (100, 3)


Iremos dividir este conjunto em entrada (X) e saída desejada (Y, target) e converter as strings "positive" e "negative" do target para valores booleanos:

In [ ]:
X_treino = treino['text']
Y_treino = treino['label']

print(f'Primeiras linhas de X_treino:\n{X_treino.head()}\n')
print(f'Primeiras linhas de Y_treino:\n{Y_treino.head()}\n')

mapeamento = {'positive': True, 'negative': False}
Y_treino = Y_treino.map(mapeamento)
Y_treino = torch.tensor(Y_treino.values, dtype=torch.long)
print(f'Tamanho de Y_treino: {Y_treino.shape}')
print(f'5 primeiras linhas de Y_treino: {Y_treino[:5]}')
print(f'Número de exemplos positivos: {(Y_treino == True).sum()}')
print(f'Número de exemplos negativos: {(Y_treino == False).sum()}')

Primeiras linhas de X_treino:
0    Un-bleeping-believable! Meg Ryan doesn't even ...
1    This is a extremely well-made film. The acting...
2    Every once in a long while a movie will come a...
3    Name just says it all. I watched this movie wi...
4    This movie succeeds at being one of the most u...
Name: text, dtype: object

Primeiras linhas de Y_treino:
0    negative
1    positive
2    negative
3    positive
4    negative
Name: label, dtype: object

Tamanho de Y_treino: torch.Size([100])
5 primeiras linhas de Y_treino: tensor([0, 1, 0, 1, 0])
Número de exemplos positivos: 51
Número de exemplos negativos: 49


# Definindo o tokenizador

Agora temos a função de tokenização, isto é, que converte strings para tokens.

In [ ]:
def tokenize(text: str):
    """
    Convert string to a list of tokens (i.e., words).
    This function lower cases everything and removes punctuation.
    """
    # Escreva aqui seu código.
    return

## Testando a função com um exemplo simples


In [ ]:
assert tokenize("I like to eat pizza.") == ['i', 'like', 'to', 'eat', 'pizza'], "Não passou no assert."
print('Passou no assert!')

AssertionError: ignored

# Definindo o vocabulário

Selecionaremos os `max_tokens` (ex: 1000) tokens mais frequentes do dataset de treino como sendo nosso vocabulário.

In [ ]:
def create_vocab(texts: List[str], max_tokens: int):
    """
    Returns a dictionary whose keys are tokens and values are token ids (from 0 to max_tokens - 1).
    """
    # Escreva aqui seu código.
    return

## Testando a função


In [ ]:
# Escreva aqui seu(s) assert(s).

# Função para converter string para Bag-of-words

In [ ]:
def convert_to_bow(text: str, vocab):
    """
    Returns a bag-of-word vector of size len(vocab).
    """
    # Escreva aqui seu código.
    return

## Testando a função

In [ ]:
# Escreva aqui seu(s) assert(s).

## Definindo a Rede Neural

**Entrada:**

$x \in R^{B \times |V|}$     (bag-of-words)

**Parametros:**

$W \in R^{|V| \times K}$    (weights: matriz de pesos)

$b \in R^{K}$    (bias/viés)

**Saída:**

$p \in R^{B \times K}$  (probabilidade de cada classe)


**Onde:**

$K$ = número de classes

$B$ = tamanho do batch

$|V|$ = tamanho do vocabulário

**Definição da rede:**

$z = xW + b$   (camada linear. $z$ é chamado de logits)

$p_i = \frac{e^{z_i}}{\sum_{j=0}^{K-1} e^{z_j}}$   (softmax)



In [ ]:
class MyModel():

    def __init__(self, dim: int):
        # Escreva seu código aqui.

    def __call__(self, x):
        # Escreva seu código aqui.
        return probs

## Testando modelo com uma entrada aleatória

Escreva abaixo um pequeno código para testar se seu modelo processa uma matriz de entrada de tamanho `batch_size, dim`, ou seja, a matriz contém `batch_size` exemplos, cada um sendo representado por um vetor de tamanho `dim`.

In [ ]:
# Escreva seu código aqui.

# Função de custo Entropia Cruzada

$y \in R^{K}$  (target),

a equação da entropia cruzada associada a um exemplo é dada por:

$L = \sum_{i=0}^{K-1} -y_i \log p_i$   (esta é a loss por exemplo)

Se $y$ for um vetor one-hot (apenas um dos elementos é diferente de zero), podemos simplicar a equação acima para:

$L = -\log p_i$

Onde $i$ é o indice da classe correta. Ou seja, $p_i$ é a probabilidade que o modelo colocou na classe correta.

A função de custo é a **média** da entropia cruzada de cada exemplo no batch.

In [ ]:
def cross_entropy_loss(probs, targets):
    """
    Args:
      probs: a float32 matrix of shape (batch_size, number of classes)
      targets: a long (int64) array of shape (batch_size)

    Returns:
      Mean loss in the batch.
    """
    # Rescreva o código abaixo sem usar laço.
    # batch_size = probs.shape[0]
    # losses = []
    # for i in range(batch_size):
    #     losses.append(-torch.log(probs[i, targets[i]]))
    #
    # losses = torch.stack(losses)
    return

## Testando a função entropia cruzada com probabilidades de 50%

Escreva abaixo um pequeno código para testar se a entropia cruzada confere com a resposta do problema 3.6 do exercício da semana passada. Crie um tensor para as probabilidades (50%) e um target também aleatório balanceado e calcule a cross entropia. Qual é o valor esperado da cross entropia nesse caso?

In [ ]:
# escreva seu código aqui

# Convertendo dataset de treino para uma matriz de bag-of-words

In [ ]:
vocab = create_vocab(X_treino, max_tokens=1000)
bows = []
for text in X_treino:
    bow = convert_to_bow(text, vocab)
    bows.append(bow)

X = torch.stack(bows)
print(X.shape)

# Laço de Treinamento

In [ ]:
num_iterations = 100
learning_rate = 0.01

model = MyModel(dim=len(vocab))

for i in range(num_iterations):
    # Zera os gradientes
    if model.weights.grad is not None:
        model.weights.grad.data.zero_()
        model.bias.grad.data.zero_()

    probs = model(X)
    loss = cross_entropy_loss(probs, Y_treino)
    print(f'iteration: {i}  loss: {loss:.6f}  exp(loss): {torch.exp(loss):.4f}')
    loss.backward()

    #Atualiza os pesos
    model.weights.data = model.weights.data - learning_rate * model.weights.grad.data
    model.bias.data = model.bias.data - learning_rate * model.bias.grad.data